# Lesson 4 - Dataset Pools

In [4]:
import os
from ansys.mapdl.core import launch_mapdl
from path_tools import *

# Define project directory and TMP folder
project_dir = os.getcwd()
tmp_dir = os.path.join(project_dir, "tmp")

# Create tmp directory if it doesn't exist
os.makedirs(tmp_dir, exist_ok=True)

# Path to the ANSYS executable
ansys_path = "C:\\Program Files\\ANSYS Inc\\v242\\ansys\\bin\\winx64\\ANSYS242.exe"

# Convert the Windows path to WSL path
ansys_path = windows_to_wsl_path(ansys_path)

# Job name
job_name = "strain_dataset"
print("Run New MAPDL instance")

mapdl = launch_mapdl(run_location=tmp_dir, 
                     remove_temp_dir_on_exit = True, 
                     cleanup_on_exit = True,
                     jobname = job_name,
                     exec_file = ansys_path,
                     additional_switches = '-smp', 
                     nproc = 2)

Run New MAPDL instance


KeyboardInterrupt: 

In [ ]:
def model(mapdl, load_x_inches, load_y_inches):    
    Length_Inches = 4.5
    Height_Inches = 2

    sensors = {'x1': 1, 'y1': .5,
                  'x2': 2.25, 'y2': .5,
                  'x3': 3.5, 'y3': .5,
                  'x4': 1, 'y4': 1.5,
                  'x5': 2.25, 'y5': 1.5,
                  'x6': 3.5, 'y6': 1.5,
                  'x7': .5, 'y7': 1,
                  'x8': 4, 'y8': 1
                 }
    sensors['load_x'] = load_x_inches
    sensors['load_y'] = load_y_inches
    
    sensors_data = {}

    mapdl.clear()
    mapdl.prep7()
    mapdl.blc4(0, 0, Length_Inches, Height_Inches, 0)
    mapdl.adele('all')

    mapdl.lfillt(1,2,.2)
    mapdl.lfillt(2,3,.2)
    mapdl.lfillt(3,4,.2)
    mapdl.lfillt(4,1,.2)

    mapdl.numcmp('area')
    mapdl.al('all')

    mapdl.mp('ex',1,1e7)
    mapdl.sectype(1,'shell')
    mapdl.secdata(.1,1)
    mapdl.et(1,181)

    for points in range(1, (int((len(sensors)-2)/2))+1):
        mapdl.hptcreate('area',1,'','coord',sensors['x'+str(points)], sensors['y'+str(points)],0)
    mapdl.hptcreate('area',1,'','coord',sensors['load_x'], sensors['load_y'],0)
    mapdl.nummrg('kp')
    
    mapdl.mshape(1,'2d')
    mapdl.smrtsize(3)
    mapdl.amesh('all')

    for points in range(1, (int((len(sensors)-2)/2))+1):
        node = mapdl.queries.node(sensors['x'+str(points)], sensors['y'+str(points)],0)
        mapdl.nrefine(node,level=2,depth=1)

    mapdl.nsel('none')
    mapdl.cm('sensors','node')
    mapdl.allsel()
    
    for points in range(1, (int((len(sensors)-2)/2))+1):
        node = mapdl.queries.node(sensors['x'+str(points)], sensors['y'+str(points)],0)
        mapdl.cmsel('s','sensors')
        mapdl.nsel('a','node','',node)
        mapdl.cm('sensors','node')
        mapdl.allsel()    

    mapdl.finish()
    mapdl.slashsolu()
    mapdl.dl('all','','all',0)

    force_node = mapdl.queries.node(sensors['load_x'], sensors['load_y'],0)
    mapdl.f(force_node,'fz',1)

    mapdl.nlgeom('on')
    solve = mapdl.solve()
    mapdl.finish()

    mapdl.post1()
    mapdl.set('last')

    mapdl.allsel()
    
    result = {}
    result["Load_X"] = sensors["load_x"]
    result["Load_Y"] = sensors["load_y"]
    for points in range(1, (int((len(sensors)-2)/2))+1):
        node = mapdl.queries.node(sensors['x'+str(points)], sensors['y'+str(points)],0)
        strain_x = mapdl.get_value('node',node,'epto','x')
        strain_y = mapdl.get_value('node',node,'epto','y')
        result["Strn_X_"+str(points)] = strain_x
        result["Strn_Y_"+str(points)] = strain_y
        
    return result

In [ ]:
inputs = []
with open('pointset.csv', newline='') as csvfile:
    point_input = csv.reader(csvfile, delimiter=',', quotechar='|')
    for point in point_input:
        inputs += [(float(point[0]),float(point[1]))]
header = {
    "Load_X":[],"Load_Y":[],"Strn_X_1":[], "Strn_Y_1":[],"Strn_X_2":[], "Strn_Y_2":[],"Strn_X_3":[], "Strn_Y_3":[],
    "Strn_X_4":[], "Strn_Y_4":[],"Strn_X_5":[], "Strn_Y_5":[],"Strn_X_6":[], "Strn_Y_6":[],"Strn_X_7":[], "Strn_Y_7":[],
    "Strn_X_8":[], "Strn_Y_8":[]
}

data_1 = pd.DataFrame(header)
output = pool.map(model, inputs, progress_bar=True, wait=True)
data_2 = pd.DataFrame(output)
data_1 = pd.concat([data_1, data_2], ignore_index=True)


In [ ]:
data_1.to_pickle("ResultDataAll.pkl")
pool.exit()